# Preliminary baseline results

In [1]:
import os
import pickle
from pathlib import Path
from functools import partial

import torch
import numpy as np
import pandas as pd
import yaml
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import Band, ColumnDataSource, NumeralTickFormatter
from bokeh.palettes import plasma, Colorblind8
output_notebook()

from liftoff.liftoff_results import collect_results
collect_results = partial(collect_results, results_dir='../results', names=['testing.pkl'])

Loading BokehJS ...

## Fetch the results paths

In [2]:
def make_key(path):
    algo = path.parts[2].split('_')[-1]
    game_name = path.parts[3][7:]
    trial = path.parts[4]
    return '-'.join([game_name, algo, trial])

In [3]:
experiments = ("two_way_dqn", "two_way_ddqn", "two_way_per")

exp_results = [collect_results(experiment_name=exp_name) for exp_name in experiments]
exp_results = [trial for exp in exp_results for trial in exp]


file_paths = [Path(f"{res[0]}/testing.pkl") for res in exp_results if res[1]]

## Preprocess it to dataframes

In [4]:
value_keys = ['ep_cnt', 'rw_per_ep', 'step_per_ep', 'rw_per_step', 'max_q', 'test_fps']
idx_keys = ['step_idx', 'time_idx']

def get_idxs(result, idx_types):
    idxs = {}
    for idx_type in idx_types:
        idxs[idx_type] = [row[idx_type] for row in result['rw_per_ep']]
    return idxs

def get_values(result, value_types):
    values = {}
    for value in value_types:
        values[value] = [row['value'] for row in result[value]]
    return values

def merge_experiments(results):
    for k, v in results.items():
        game, algo, trial = k.split('-')
        v['game'] = game
        v['algo'] = algo
        v['trial'] = trial
    return pd.concat(results.values(), ignore_index=True, sort=False)

def make_df(results_dict):
    # make dataframes out of the dictinaries for each experiment
    results = {key: pd.DataFrame(data = {**get_idxs(vals, idx_keys), **get_values(vals, value_keys)})
               for key, vals in results_dict.items()}
    
    # merge the experiments, adding `variant` and `trial` columns
    return merge_experiments(results)

## Visualize the results

In [5]:
def transform(data, groupby, y, transform='mean'):
    if transform == 'mean':
        return data.groupby(groupby)[y].mean().to_frame().reset_index()
    elif transform == 'std':
        return data.groupby(groupby)[y].std().to_frame().reset_index()
    else:
        raise ValueError


def set_figure(title, y_axis_scale='linear', y_axis_format=".00"):
    fig = figure(title=title, plot_height=500, plot_width=950,
                  y_axis_type=y_axis_scale, background_fill_color='#ffffff')
    fig.yaxis.formatter=NumeralTickFormatter(format=y_axis_format)
    
    if y_axis_scale == 'log':
        fig.ygrid.minor_grid_line_color = '#5B5B5B'
        fig.ygrid.minor_grid_line_alpha = 0.1
    
    return fig


def add_trials(data, x, y, event, trials, event_name, fig, color, aggregate):
    dff = data.loc[data[event] == event_name]
    
    trial_names = dff[trials].unique()
    
    alpha = 1 if not aggregate else 0.7
    alpha_stop = 0.2
    alpha_step = (alpha - alpha_stop) / (len(trial_names) - 1)
    
    lw = 3 if not aggregate else 2
    
    for trial_name in trial_names:
        df = dff.loc[dff[trials] == trial_name]
        df = df.sort_values(by=[x]).reset_index(drop=True)
        fig.line(x=x, y=y, legend=event_name, source=df, line_width=lw, color=color, alpha=alpha)
        alpha -= alpha_step
    return fig


def add_band(x, y, hue_mean, hue_var, hue_name, fig, color):
    hue_var['lower'] = hue_mean[y] - hue_var[y]
    hue_var['upper'] = hue_mean[y] + hue_var[y]
    
    # Bollinger shading glyph:
    band_x = np.append(hue_var[x].values, hue_var[x].values[::-1])
    band_y = np.append(hue_var['lower'].values, hue_var['upper'].values[::-1])
    
    fig.patch(x=band_x, y=band_y, legend=hue_name, color=color, alpha=0.3)
    
    return fig


def lineplot(data, x, y, hue=None, trials=None, aggregate=True, legend_pos='top_left', y_axis_format="00.00",
             title="Plot", y_axis_scale='linear'):
    # set the figure
    fig = set_figure(title, y_axis_format=y_axis_format, y_axis_scale=y_axis_scale)        
    
    # get the mean of each event
    y_mean = transform(data, [x, hue], y)
    
    # get the names of each event we're plotting
    hues = y_mean[hue].unique()
    palette = plasma(len(hues))
    
    # iterate through events and create a line for each
    for hue_name, color in zip(hues, palette):
        
        hue_mean = y_mean.loc[y_mean[hue] == hue_name]
        if aggregate:
            fig.line(x=x, y=y, legend=hue_name, source=hue_mean, line_width=4, color=color, alpha=1)
        
        if trials:
            fig = add_trials(data, x, y, hue, trials, hue_name, fig, color, aggregate)
        else:
            y_var = transform(data, [x, hue], y, transform='std')
            hue_var = y_var.loc[y_var[hue] == hue_name].copy()
            fig = add_band(x, y, hue_mean, hue_var, hue_name, fig, color)
    
    # additional settings
    fig.legend.location = legend_pos
    fig.legend.click_policy="hide"
    return fig


def simple_plot(data, x, y, trials):
    fig = set_figure(title="Simple Plot", y_axis_format='0.0000[00]')
    
    trial_vals = data[trials].unique()
    trial_names = [f'trial {trial_id}' for trial_id in trial_vals]

    if len(Colorblind8) < len(trial_vals):
        palette = plasma(len(trials))
    else:
        palette = Colorblind8

    for trial, color, legend in zip(trial_vals, palette, trial_names):
        df = data.loc[data[trials] == trial]
        fig.line(x=df[x], y=df[y], legend=legend, line_width=4, color=color, alpha=1)
    
    # additional settings
    fig.legend.click_policy="hide"
    fig.legend.location = "top_left"
    return fig

### Some data utils

In [6]:
def filter_by(game, algo=None):
    if algo:
        return data.loc[(data['game'] == game) & (data['algo'].isin(algo))]
    return data.loc[data['game'] == game]


def smooth_data(data):
    trials = []
    for trial_value in data['trial'].unique():
        trial = data.loc[data['trial'] == trial_value].reset_index()
        trial['roll'] = trial['rw_per_ep'].rolling(window=5, min_periods=1).mean()
        trials.append(trial)
    return pd.concat(trials, ignore_index=True, sort=False)

## Load data

In [7]:
results_dict = {make_key(path): pickle.load(open(path, 'rb')) for path in file_paths}
for k in results_dict.keys():
    print(k)
    
data = make_df(results_dict)
data.sample(15)

seaquest-dqn-0
road_runner-dqn-2
road_runner-dqn-0
road_runner-dqn-1
seaquest-ddqn-0
seaquest-ddqn-1
road_runner-ddqn-2
road_runner-ddqn-1
seaquest-per-2
seaquest-per-0
seaquest-per-1
road_runner-per-2
road_runner-per-0
road_runner-per-1


,step_idx,time_idx,ep_cnt,rw_per_ep,step_per_ep,rw_per_step,max_q,test_fps,game,algo,trial
135,5500000,25061.592763,100,20969.0,911.48,23.005442,16.017584,39.485759,road_runner,ddqn,2
256,10500000,48462.491584,100,5134.9,3261.33,1.574480,37.992393,125.569827,seaquest,per,0
221,11500000,53800.178450,100,4271.3,2763.75,1.545473,35.738945,110.859963,seaquest,per,2
187,13000000,66857.729038,100,7179.0,662.70,10.832956,16.035542,28.442928,road_runner,ddqn,1
5,3000000,12962.688241,100,367.0,1178.81,0.311331,21.577820,53.207491,seaquest,dqn,0
186,12500000,64527.799852,100,0.0,10300.13,0.000000,5.708809,225.398166,road_runner,ddqn,1
213,7500000,34252.735864,100,3380.3,2849.83,1.186141,29.596409,114.809863,seaquest,per,2
66,7000000,30832.875181,100,0.0,167.66,0.000000,54.102139,8.025287,road_runner,dqn,0
173,6000000,27807.013982,100,10450.0,1024.44,10.200695,14.348265,42.734353,road_runner,ddqn,1
334,1000000,4311.160368,100,5756.0,1014.99,5.670992,4.843754,41.930171,road_runner,per,1


# Seaquest

In [8]:
show(lineplot(filter_by('seaquest', ['per', 'ddqn']),
              'step_idx', 'rw_per_ep', 'algo', 'trial', aggregate=True, title="Seaquest DDQN vs PER"))

### Smoothed Seaquest

In [9]:
show(lineplot(smooth_data(filter_by('seaquest', ['per'])),
              'step_idx', 'roll', 'algo', 'trial', aggregate=True, title="Smoothed Seaquest PER"))

## RoadRunner

In [10]:
show(lineplot(filter_by('road_runner'),
              'step_idx', 'rw_per_ep', 'algo', 'trial', aggregate=True, title="RoadRunner - DQN vs DDQN vs PER"))

In [11]:
show(lineplot(smooth_data(filter_by('road_runner', ['ddqn'])),
              'step_idx', 'roll', 'algo', 'trial', aggregate=True, title="Smoothed RoadRunner - DDQN"))

In [12]:
show(lineplot(smooth_data(filter_by('road_runner', ['dqn'])),
              'step_idx', 'roll', 'algo', 'trial', aggregate=True, title="Smoothed RoadRunner - DQN"))

In [13]:
show(lineplot(smooth_data(filter_by('road_runner', ['per'])),
              'step_idx', 'roll', 'algo', 'trial', aggregate=True, title="Smoothed RoadRunner - PER"))

In [14]:
# show(simple_plot(data.loc[data['game'] == 'seaquest'], 'step_idx', 'rw_per_ep', 'trial'))

In [15]:
# road_runner_dqn = data.loc[(data['game'] == 'road_runner') & (data['algo'] == 'ddqn')]
# show(simple_plot(road_runner_dqn, 'step_idx', 'rw_per_ep', 'trial'))